In [ ]:
class CarPricePredictor:
    def __init__(self, model_path):
        self.model = CatBoostRegressor()
        self.model.load_model(model_path)
        self.label_encoder = None
        self.scaler = None
    
    def prepare_data(self, vin):
        # Проверяем валидность VIN-кода
        if not self.is_valid_vin(vin):
            print("Невалидный VIN-код")
            return None
        
        # Преобразуем VIN-код в соответствующие признаки
        vin_features = {
            'WMI_country': vin[0],
            'WMI_manufacturer': vin[1:3],
            'VDS_model': vin[3:6],
            'VDS_body_type': vin[6],
            'VDS_engine_type': vin[7],
            'VDS_check_digit': vin[8],
            'year_of_manufacture': vin[9],
            'VIS_serial_number': vin[10:]
        }
        
        # Создаем DataFrame из признаков
        df = pd.DataFrame([vin_features])
        
        # Применяем Label Encoding к категориальным признакам
        if self.label_encoder is not None:
            for feature in df.columns:
                if feature in self.label_encoder.classes_:
                    df[feature] = self.label_encoder.transform(df[feature])
        
        # Масштабируем признаки
        if self.scaler is not None:
            df = self.scaler.transform(df)
        
        return df
    
    def predict_price(self, vin):
        # Подготовка данных для предсказания
        data = self.prepare_data(vin)
        if data is None:
            return None
        
        # Предсказание стоимости автомобиля
        return self.model.predict(data)[0]
    
    @staticmethod
    def is_valid_vin(vin):
        if len(vin) != 17:
            return False
        if not re.match(r'^[A-HJ-NPR-Z0-9]+$', vin):
            return False

        transliteration = "0123456789.ABCDEFGH..JKLMN.P.R..STUVWXYZ"
        weights = [8, 7, 6, 5, 4, 3, 2, 10, 0, 9, 8, 7, 6, 5, 4, 3, 2]
        checksum = 0
        for i in range(17):
            if transliteration.find(vin[i]) == -1:
                return False
            checksum += weights[i] * (transliteration.find(vin[i]) % 10)
        if vin[8].isdigit():
            expected = int(vin[8])
        elif vin[8] == 'X':
            expected = 10
        else:
            return False
        return (checksum % 11) == expected

In [ ]:

# Пример использования
predictor = CarPricePredictor(model_path='catboost_model.cbm')
vin = '1FAHP2E89DG119368'
predicted_price = predictor.predict_price(vin)
if predicted_price is not None:
    print(f"Предсказанная стоимость автомобиля с VIN-кодом {vin}: {predicted_price}")